In [1]:
from torchvision import utils
from basic_fcn import *
from dataloader import *
from utils import *
import torchvision
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import time
from matplotlib import pyplot as plt
import os
from pprint import pprint

In [2]:
train_dataset = CityScapesDataset(csv_file='./Data/train.csv')
val_dataset = CityScapesDataset(csv_file='./Data/val.csv')
test_dataset = CityScapesDataset(csv_file='./Data/test.csv')

_batch_size = 8
_num_workers = 4
iou_classes = [11, 20, 24, 26, 33]

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=_batch_size,
                          num_workers=_num_workers,
                          shuffle=True)
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=_batch_size,
                          num_workers=_num_workers,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=_batch_size,
                          num_workers=_num_workers,
                          shuffle=True)

In [3]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        #torch.nn.init.xavier_uniform(m.bias.data)
        m.bias.data.zero_()
        
epochs     = 100
criterion = nn.CrossEntropyLoss()# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
fcn_model = FCN(n_class=n_class)
# for name, params in fcn_model.named_parameters():
#     print(name, params.shape, params.requires_grad)
    
fcn_model.apply(init_weights)
#fcn_model = torch.load('best_model')

optimizer = optim.Adam(fcn_model.parameters(), lr=5e-3)

In [4]:
modelName = 'fcn'
MYDIR = "./Results" + "/" + modelName
CHECK_FOLDER = os.path.isdir(MYDIR)
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)
else:
    print(MYDIR, "folder already exists.")

def plotTwoGraphs(base, train_metrics, val_metrics):
    plt.figure()
    plt.plot(base, train_metrics)
    plt.plot(base, val_metrics)
    plt.gca().legend(('train','validation'))
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss vs Epochs')
    plt.savefig(MYDIR + '/loss.png')

def plotGraph(base, metrics, name):
    plt.figure()
    plt.plot(base, metrics)
    plt.xlabel('Epochs')
    plt.ylabel(name)
    plt.title(name + ' vs Epochs')
    plt.savefig(MYDIR + '/' + name + '.png')
    

./Results/fcn folder already exists.


In [ ]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    fcn_model = fcn_model.cuda()
    
all_train_losses = []
all_val_losses = []
all_val_accuracies = []
all_val_iou = []
all_val_iou_class = {}


        
def train():
    startingValLoss = 10000000.0
    for epoch in range(epochs):
        fcn_model.train()
        curEpochLoss = []
        ts = time.time()
        for iter, (X, tar, Y) in enumerate(train_loader):
            optimizer.zero_grad()

            if use_gpu:
                inputs = X.cuda()   # Move your inputs onto the gpu
                labels = Y.cuda() # Move your labels onto the gpu
            else:
                inputs, labels = X, Y# Unpack variables into inputs and labels
            outputs = fcn_model(inputs)
            loss = criterion(outputs, labels)
            curEpochLoss.append(loss.item())
            loss.backward()
            optimizer.step()

            if iter % 100 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item(), time.time() - ts))
        curEpochLoss = sum(curEpochLoss)/len(curEpochLoss)
        print('training loss : ', curEpochLoss)
        all_train_losses.append(curEpochLoss)
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        
        
        val_loss, val_accuracy, val_iou_net, val_iou_net_class_wise = val(epoch)
        all_val_losses.append(val_loss)
        all_val_accuracies.append(val_accuracy)
        all_val_iou.append(val_iou_net)
        for c in val_iou_net_class_wise:
            if(c not in all_val_iou_class):
                all_val_iou_class[c] = []
            all_val_iou_class[c].append(val_iou_net_class_wise[c])
        
        plotTwoGraphs([i for i in range(epoch+1)], all_train_losses, all_val_losses)
        plotGraph([i for i in range(epoch+1)], all_val_accuracies, 'validation_accuracy')
        plotGraph([i for i in range(epoch+1)], all_val_iou, 'average_iou')
        for c in all_val_iou_class:
            plotGraph([i for i in range(epoch+1)], all_val_iou_class[c], 'iou'+str(c))
        
        if(val_loss < startingValLoss):
            startingValLoss = val_loss
            results = {}
            results['accuracy'] = val_accuracy
            results['iou_net'] = val_iou_net
            results['iou_net_class_wise'] = val_iou_net_class_wise
            results['total_loss'] = val_loss
            results['epoch'] = epoch
            with open(MYDIR + '/results_val.txt', 'wt') as out:
                pprint(results, stream=out)
            torch.save(fcn_model, MYDIR + '/best_model')
            #test(epoch)
    


def val(epoch):
    fcn_model.eval()
    #Complete this function - Calculate loss, accuracy and IoU for every epoch
    # Make sure to include a softmax after the output from your model
    total_loss = []
    acc_num = 0
    acc_den = 0
    #iou_num = len(iou_classes)*[0] 
    #iou_den = len(iou_classes)*[0] 
    iou_num = {}
    iou_den = {}
    
    for iter, (X_val, tar_val, Y_val) in enumerate(val_loader):
        if use_gpu:
            inputs_val = X_val.cuda()   # Move your inputs onto the gpu
            labels_val = Y_val.cuda() # Move your labels onto the gpu
        else:
            inputs_val, labels_val = X_val, Y_val# Unpack variables into inputs and labels

        outputs_val = fcn_model(inputs_val)
        
        _, preds = torch.max(outputs_val, 1)
        
        loss_val = criterion(outputs_val, labels_val).item()
        
        preds_cpu = preds.cpu()
        Y_cpu = Y_val.cpu()
        
        total_loss.append(loss_val)
        
        nn, dd = pixel_acc(preds_cpu, Y_cpu)
        acc_num += nn
        acc_den += dd
        
        for iou_cc_idx in range(len(iou_classes)):
            i_nn, i_dd = iou(preds_cpu, Y_cpu, iou_classes[iou_cc_idx])
            if iou_cc_idx not in iou_num:
                iou_num[iou_cc_idx] = 0
                iou_den[iou_cc_idx] = 0
            iou_num[iou_cc_idx] += i_nn
            iou_den[iou_cc_idx] += i_dd
            
        del inputs_val
        del labels_val
        del loss_val
    total_loss = sum(total_loss)/len(total_loss)
    print("Validation loss after epoch {}, , loss: {}".format(epoch, total_loss))
    print("Accuracy on val after epoch {}, accuracy: {}".format(epoch, 100*acc_num/acc_den))
#     for iou_cc_idx in range(len(iou_classes)):
#         print("IOU for class {} on val after epoch{}, iou: {}".format(iou_classes[iou_cc_idx], epoch, 100*iou_num[iou_cc_idx]/iou_den[iou_cc_idx]))

    iou_net_class_wise = dict((c, iou_num[c]/iou_den[c]) for c in iou_num)
    accuracy = acc_num / acc_den
    iou_net_num = 0
    iou_net_den = 0
    for c in iou_num:
        iou_net_num += iou_num[c]
        iou_net_den += iou_den[c]
    iou_net = iou_net_num/iou_net_den

    return total_loss, accuracy, iou_net, iou_net_class_wise
    
def test(epoch):
    fcn_model.eval()
    #Complete this function - Calculate loss, accuracy and IoU for every epoch
    # Make sure to include a softmax after the output from your model
    total_loss = 0
    acc_num = 0
    acc_den = 0
#     iou_num = len(iou_classes)*[0] 
#     iou_den = len(iou_classes)*[0]
    iou_num = {}
    iou_den = {}
    
    for iter, (X_val, tar_val, Y_val) in enumerate(test_loader):
        if use_gpu:
            inputs_val = X_val.cuda()   # Move your inputs onto the gpu
            labels_val = Y_val.cuda() # Move your labels onto the gpu
        else:
            inputs_val, labels_val = X_val, Y_val# Unpack variables into inputs and labels

        outputs_val = fcn_model(inputs_val)
        
        _, preds = torch.max(outputs_val, 1)
        
        loss_val = criterion(outputs_val, labels_val).item()
        
        preds_cpu = preds.cpu()
        Y_cpu = Y_val.cpu()
        
        total_loss += loss_val
        
        nn, dd = pixel_acc(preds_cpu, Y_cpu)
        acc_num += nn
        acc_den += dd
        
        for iou_cc_idx in range(len(iou_classes)):
            i_nn, i_dd = iou(preds_cpu, Y_cpu, iou_classes[iou_cc_idx])
            if iou_cc_idx not in iou_num:
                iou_num[iou_cc_idx] = 0
                iou_den[iou_cc_idx] = 0
            iou_num[iou_cc_idx] += i_nn
            iou_den[iou_cc_idx] += i_dd
            
        del inputs_val
        del labels_val
        del loss_val
    print((acc_num, acc_den, iou_num, iou_den, total_loss))
        
    print("Test loss after epoch {}, , loss: {}".format(epoch, total_loss))
    print("Accuracy on test after epoch {}, accuracy: {}".format(epoch, ''))
#     for iou_cc_idx in range(len(iou_classes)):
#         print("IOU for class {} on val after epoch{}, iou: {}".format(iou_classes[iou_cc_idx], epoch, 100*iou_num[iou_cc_idx]/iou_den[iou_cc_idx]))

    iou_net_class_wise = dict((c, iou_num[c]/iou_den[c]) for c in iou_num)
    for c in iou_num:
        try:
            iou_net_class_wise[c] = iou_num[c]/iou_den[c]
        except:
            iou_net_class_wise[c] = 0.0
        
    try:
        accuracy = acc_num / acc_den
    except:
        accuracy = 0
        
    iou_net_num = 0
    iou_net_den = 0
    for c in iou_num:
        iou_net_num += iou_num[c]
        iou_net_den += iou_den[c]
    try:
        iou_net = iou_net_num/iou_net_den
    except:
        iou_net = 0.0
    results = {}
    results['accuracy'] = accuracy
    results['iou_net'] = iou_net
    results['iou_net_class_wise'] = iou_net_class_wise
    results['total_loss'] = total_loss
    results['epoch'] = epoch
    print('yayyy, code reached till here!')
    with open(MYDIR + '/results_test.txt', 'wt') as out:
        pprint(results, stream=out)
              
    
if __name__ == "__main__":
    test(0)
    train()

(0, 0, {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}, {0: 15228281, 1: 1040357, 2: 5532990, 3: 621145, 4: 9056313}, 673.5386364459991)
Test loss after epoch 0, , loss: 673.5386364459991
Accuracy on test after epoch 0, accuracy: 
yayyy, code reached till here!
epoch0, iter0, loss: 3.9734296798706055
epoch0, iter100, loss: 2.594252347946167
epoch0, iter200, loss: 1.7255687713623047
epoch0, iter300, loss: 2.404722213745117
training loss :  1.9123156975994828
Finish epoch 0, time elapsed 97.90253400802612
Validation loss after epoch 0, , loss: 1.8384951986963787
Accuracy on val after epoch 0, accuracy: 55.39011873433441


/opt/conda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FCN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.7/site-pa

epoch1, iter0, loss: 1.6959939002990723
epoch1, iter100, loss: 2.809593677520752
epoch1, iter200, loss: 1.3752617835998535
epoch1, iter300, loss: 2.2059478759765625
training loss :  1.621572133834644
Finish epoch 1, time elapsed 95.41624569892883
Validation loss after epoch 1, , loss: 1.5998298573115515
Accuracy on val after epoch 1, accuracy: 60.74135920177723
epoch2, iter0, loss: 1.2041064500808716
epoch2, iter100, loss: 1.9458788633346558
epoch2, iter200, loss: 1.2404119968414307
epoch2, iter300, loss: 1.8474711179733276
training loss :  1.5758285370244776
Finish epoch 2, time elapsed 98.28761148452759
Validation loss after epoch 2, , loss: 1.4063781726927984
Accuracy on val after epoch 2, accuracy: 67.35423389088349


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


epoch3, iter0, loss: 1.1595466136932373
epoch3, iter100, loss: 2.0788395404815674
